In [ ]:
#Importing modules
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import * 
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler


Reading data

In [ ]:
video_data=pd.read_csv("/content/drive/MyDrive/video_all.csv")

In [ ]:
video_data.rename(columns={"25088":"Filename"},inplace=True)
video_data.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
audio_data=pd.read_csv("/content/drive/MyDrive/audio_all.csv")

In [ ]:
audio_data.drop("Unnamed: 0",axis=1,inplace=True)
audio_data.rename(columns={"Name":"Filename"},inplace=True)

In [ ]:
video_data['Filename'] = video_data['Filename'].str.replace(".mp4","")
audio_data['Filename'] = audio_data['Filename'].str.replace(".csv","")

In [ ]:
labels_data=pd.read_csv("/content/drive/MyDrive/labels.csv")

In [ ]:
labels_data.drop(["Start","End","Score"],axis=1,inplace=True)

In [ ]:
v_label = pd.merge(video_data, labels_data,how="outer", on="Filename")
a_label=pd.merge(audio_data, labels_data,how="outer", on="Filename")

Data cleaning on Audio data

In [ ]:
for x in a_label.columns:
  if a_label[x].dtypes == "O":
    for i in range (len(a_label[x])):
      if "."  in a_label[x].iloc[i][3:]:
        a_label[x].iloc[i] = 0

for x in a_label.columns:
  try:
    if a_label[x].dtypes == "O":
      a_label[x] = a_label[x].astype(float)
  except:
    pass

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Data Preprocessing

In [ ]:
v_label.replace("positive",1,inplace=True)
v_label.replace("negative",0,inplace=True)
a_label.replace("positive",1,inplace=True)
a_label.replace("negative",0,inplace=True)

In [ ]:
v_label.drop(["Filename"],axis=1,inplace=True)
a_label.drop(["Filename"],axis=1,inplace=True)

In [ ]:
v_label

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,25050,25051,25052,25053,25054,25055,25056,25057,25058,25059,25060,25061,25062,25063,25064,25065,25066,25067,25068,25069,25070,25071,25072,25073,25074,25075,25076,25077,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087,Sentiment,Type
0,0.368188,0.0,0.0,0.053985,1.029262,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.580487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.424640,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.328400,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.076081,0.0,0.0,0.0,0.066414,0.0,0.000000,0.0,0.0,0.780245,0.0,0.0,0.434905,0.000368,0.000000,0.000000,0.000000,1.054962,0.256338,0.0,1.388605,0.158964,0.764717,0.000,0.000000,0.220051,0.0,0.000000,0.883657,0.0,1,train
1,0.445311,0.0,0.0,0.107008,1.042036,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008242,0.0,0.549145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.584536,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.009092,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.306930,0.0,0.000000,0.0,0.0,0.620191,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.760911,0.546303,0.0,1.195126,0.080785,1.166634,0.000,0.023243,0.011094,0.0,0.000000,0.750355,0.0,0,train
2,0.286750,0.0,0.0,0.054761,0.979557,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078594,0.0,0.577882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.484665,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.110471,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.685133,0.0,0.0,1.012893,0.000000,0.000000,0.000000,0.000000,1.350049,0.421928,0.0,1.454241,0.013818,0.582747,0.000,0.000000,0.000000,0.0,0.000000,0.912780,0.0,0,train
3,0.283418,0.0,0.0,0.025360,0.816509,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183081,0.0,0.542183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518735,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.149514,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.788070,0.0,0.0,0.378314,0.000000,0.000000,0.000000,0.000000,0.962997,0.519956,0.0,1.297714,0.199440,0.637510,0.000,0.000000,0.101909,0.0,0.124806,0.903690,0.0,1,train
4,0.375979,0.0,0.0,0.205068,1.332773,0.0,0.043707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.097992,0.0,0.502211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.660352,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.040291,0.0,0.0,0.0,0.0,0.0,...,0.024580,0.0,0.0,0.000000,0.0,0.0,0.048816,0.0,0.119116,0.0,0.0,0.0,0.120782,0.0,0.000000,0.0,0.0,0.776301,0.0,0.0,0.335960,0.000000,0.000000,0.000000,0.000000,0.854756,0.085146,0.0,1.230606,0.017993,0.548047,0.004,0.000000,0.000000,0.0,0.000000,0.810676,0.0,1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,0.015542,0.0,0.0,0.000000,0.230801,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009510,0.0,0.733093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.335197,0.045750,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,1.099095,0.0,0.0,0.097105,0.0,0.0,0.112383,0.0,0.438299,0.0,0.0,0.0,0.538885,0.0,0.063370,0.0,0.0,0.385225,0.0,0.0,0.571690,0.000000,0.474496,0.367230,0.103143,0.784495,0.488715,0.0,1.062415,0.081832,0.760942,0.000,0.000000,0.286278,0.0,0.000000,1.078807,0.0,1,valid
2195,0.025665,0.0,0.0,0.000000,0.185971,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022772,0.0,0.741327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.330909,0.032758,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.456603,0.0,0.0,0.000000,0.0,0.0,0.303488,0.0,0.105600,0.0,0.0,0.0,0.000000,0.0,0.222552,0.0,0.0,0.728499,0.0,0.0,0.492576,0.067412,0.157502,0.040923,0.000000,0.954354,0.445082,0.0,1.501844,0.000000,0.601123,0.000,0.000000,1.178

Normalization

In [ ]:
#Normalizing Video dataset
vmms= MinMaxScaler()
nv = v_label.drop("Type",axis=1)
typ=v_label["Type"]
vmms.fit_transform(nv)
v_label=pd.DataFrame(data=nv)
v_label["Type"] = typ

In [ ]:
#Normalizing Audio dataset
amms= MinMaxScaler()
na = a_label.drop("Type",axis=1)
typ=a_label["Type"]
amms.fit_transform(na)
a_label=pd.DataFrame(data=na)
a_label["Type"] = typ

Train/Validation/Test Split

In [ ]:
v_train  = v_label[v_label["Type"]=="train"]
v_valid = v_label[v_label["Type"]=="valid"]
v_test  = v_label[v_label["Type"]=="test"]

a_train  = a_label[a_label["Type"]=="train"]
a_valid = a_label[a_label["Type"]=="valid"]
a_test  = a_label[a_label["Type"]=="test"]


In [ ]:
v_train.drop(["Type"],axis=1,inplace=True)
v_valid.drop(["Type"],axis=1,inplace=True)
v_test.drop(["Type"],axis=1,inplace=True)
a_train.drop(["Type"],axis=1,inplace=True)
a_valid.drop(["Type"],axis=1,inplace=True)
a_test.drop(["Type"],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
v_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,25049,25050,25051,25052,25053,25054,25055,25056,25057,25058,25059,25060,25061,25062,25063,25064,25065,25066,25067,25068,25069,25070,25071,25072,25073,25074,25075,25076,25077,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087,Sentiment
0,0.368188,0.0,0.0,0.053985,1.029262,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.580487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.424640,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.328400,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076081,0.000000,0.0,0.0,0.066414,0.0,0.000000,0.0,0.0,0.780245,0.000000,0.0,0.434905,0.000368,0.0,0.0,0.0,1.054962,0.256338,0.0,1.388605,0.158964,0.764717,0.000,0.000000,0.220051,0.0,0.000000,0.883657,0.0,1
1,0.445311,0.0,0.0,0.107008,1.042036,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.008242,0.0,0.549145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.584536,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.009092,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.306930,0.0,0.000000,0.0,0.0,0.620191,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.760911,0.546303,0.0,1.195126,0.080785,1.166634,0.000,0.023243,0.011094,0.0,0.000000,0.750355,0.0,0
2,0.286750,0.0,0.0,0.054761,0.979557,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.078594,0.0,0.577882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.484665,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.110471,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.685133,0.000000,0.0,1.012893,0.000000,0.0,0.0,0.0,1.350049,0.421928,0.0,1.454241,0.013818,0.582747,0.000,0.000000,0.000000,0.0,0.000000,0.912780,0.0,0
3,0.283418,0.0,0.0,0.025360,0.816509,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.183081,0.0,0.542183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518735,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.149514,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.788070,0.000000,0.0,0.378314,0.000000,0.0,0.0,0.0,0.962997,0.519956,0.0,1.297714,0.199440,0.637510,0.000,0.000000,0.101909,0.0,0.124806,0.903690,0.0,1
4,0.375979,0.0,0.0,0.205068,1.332773,0.0,0.043707,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.097992,0.0,0.502211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.660352,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.040291,0.0,0.0,0.0,0.0,0.0,...,0.0,0.024580,0.0,0.0,0.0,0.0,0.0,0.048816,0.0,0.119116,0.000000,0.0,0.0,0.120782,0.0,0.000000,0.0,0.0,0.776301,0.000000,0.0,0.335960,0.000000,0.0,0.0,0.0,0.854756,0.085146,0.0,1.230606,0.017993,0.548047,0.004,0.000000,0.000000,0.0,0.000000,0.810676,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,0.101049,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.422325,0.0,0.0,0.017510,0.0,0.594206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675128,0.125795,0.0,0.0,0.0,0.0,0.0,0.0,0.177286,0.0,0.0,0.0,0.0,0.0,...,0.0,0.490349,0.0,0.0,0.0,0.0,0.0,0.316973,0.0,0.555487,0.033364,0.0,0.0,0.184245,0.0,0.010604,0.0,0.0,0.716890,0.109060,0.0,0.729519,0.037244,0.0,0.0,0.0,0.521728,1.006795,0.0,1.882352,0.100316,0.329153,0.000,0.237724,0.374185,0.0,0.089939,0.831598,0.0,0
1979,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.010548,0.0,0.0,0.0,0.0,0.0,0.662415,0.0,0.0,0.000000,0.0,0.742006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.632799,0.116592,0.0,0.0,0.0,0.0,0.0,0.0,0.202034,0.0,0.0,0.0,0.0,0.0,...,0.0,0.501891,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.082627,0.150969,0.0,0.0,0.244541,0.0,0.035067,0.0,0.0,0.551470,0.000000,0.0,0.465266,0.000000,0.0,0.0,0.0,0.896449,0.921306,0.0,1.704333,0.039561,0.177653,0.000,0.000000,0.206215,0.0,0.044884,0.820038,0.0,1
1980,0.000

In [ ]:


y_train = v_train["Sentiment"]
y_valid = v_valid["Sentiment"]
y_test = v_test["Sentiment"]

v_train_x = v_train.drop("Sentiment",axis=1)
v_valid_x = v_valid.drop("Sentiment",axis=1)
v_test_x = v_test.drop("Sentiment",axis=1)

a_train_x = a_train.drop("Sentiment",axis=1)
a_valid_x = a_valid.drop("Sentiment",axis=1)
a_test_x = a_test.drop("Sentiment",axis=1)



In [ ]:
#Converting dataframe objects into np array so that they can be trained in keras models

y_array_train=np.asarray(y_train).astype(np.float32)
y_array_valid=np.asarray(y_valid).astype(np.float32)
y_array_test=np.asarray(y_test).astype(np.float32)


visual_train_x_array=np.asarray(v_train_x).astype(np.float32)
audio_train_x_array=np.asarray(a_train_x).astype(np.float32)

visual_valid_x_array=np.asarray(v_valid_x).astype(np.float32)
audio_valid_x_array=np.asarray(a_valid_x).astype(np.float32)

visual_test_x_array=np.asarray(v_test_x).astype(np.float32)
audio_test_x_array=np.asarray(a_test_x).astype(np.float32)

In [ ]:
#Videos sub-model
visual_sm = Sequential()
visual_sm.add(Dense(1024, input_dim=25088 ))
visual_sm.add(Dense(512, activation='relu'))


In [ ]:
#Audios sub-model

audio_sm = Sequential()
audio_sm.add(Dense(1024, input_dim=88))
audio_sm.add(Dense(1024, activation='relu'))
audio_sm.add(Dense(512, activation='relu'))



In [ ]:
#Model Concatenation

model_concat = concatenate([visual_sm.output, audio_sm.output], axis=-1)
model_concat=Dense(1024, activation='LeakyReLU')(model_concat)
model_concat=Dense(512, activation='LeakyReLU')(model_concat)

model_concat = Dense(1, activation='sigmoid')(model_concat)
model = Model(inputs=[visual_sm.input, audio_sm.input], outputs=model_concat)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])


In [ ]:
model.fit([v_train_x, a_train_x], y_array_train,batch_size=10,steps_per_epoch=100,epochs=50,
              verbose=True)

Epoch 1/50
100/100 [==============================] - 3s 18ms/step - loss: 31.3794 - binary_accuracy: 0.5040
Epoch 2/50
100/100 [==============================] - 2s 17ms/step - loss: 1.0254 - binary_accuracy: 0.5734
Epoch 3/50
100/100 [==============================] - 2s 17ms/step - loss: 0.7433 - binary_accuracy: 0.6539
Epoch 4/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6694 - binary_accuracy: 0.6429
Epoch 5/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6499 - binary_accuracy: 0.6640
Epoch 6/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6302 - binary_accuracy: 0.6600
Epoch 7/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6084 - binary_accuracy: 0.6811
Epoch 8/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6007 - binary_accuracy: 0.7082
Epoch 9/50
100/100 [==============================] - 2s 17ms/step - loss: 0.6015 - binary_accuracy: 0.6970
Epoch 10/50
100/100 [======

In [ ]:
y_pred= model.predict([v_valid_x,a_valid_x])


In [ ]:
for x in range(len(y_pred)):
  if y_pred[x][0]>=0.5:
    y_pred[x][0]=1
  else:
    y_pred[x][0]=0

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_array_valid,y_pred)

0.620537383939363


In [ ]:
y_test_predict=model.predict([v_test_x,a_test_x])
for x in range(len(y_test_predict)):
  if y_test_predict[x][0]>=0.5:
    y_test_predict[x][0]=1
  else:
    y_test_predict[x][0]=0

In [ ]:
accuracy_score(y_array_test,y_test_predict)

0.579838276456312
